# t-Distributed Stochastic Neighbor Embedding (TSNE)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.manifold import TSNE

In [ ]:
# constants
PATH = '../data/csv/'

## Import Data

## Preprocess Data

## Dimensionality Reduction

## Visualize